- [counter.clsp](counter.clsp)
- [singleton_terminal.clsp](singleton_terminal.clsp)
- [singleton_helpers_v1_1.py](../singleton_helpers_v1_1.py)
- [singleton_top_layer_v1_1.py](https://github.com/Chia-Network/chia-blockchain/blob/1.4.0/chia/wallet/puzzles/singleton_top_layer_v1_1.py)
- [singleton_top_layer_v1_1.clvm](https://github.com/Chia-Network/chia-blockchain/blob/1.4.0/chia/wallet/puzzles/singleton_top_layer_v1_1.clvm)

In [1]:
%%bash
chia version
cdv --version

1.4.0
cdv, version 1.0.8


In [2]:
import sys
sys.path.insert(0, ".")
import singleton_counter
sys.path.insert(0, "..")
import singleton_helpers_v1_1

counter_puzzle = singleton_counter.load_program("counter.clsp", ".")
singleton_terminal_puzzle =  singleton_counter.load_program("singleton_terminal.clsp", ".")
print(counter_puzzle.get_tree_hash())
print(singleton_terminal_puzzle.get_tree_hash())

d5b0cadac424ef077dc8eacc449d0bb8544639cc39f5462a48a8d6b5750a4483
efafe67593029270221b07866b7e4b8dec7226ebd6be177ed4c9051c9ce1cc96


In [3]:
from chia.wallet.puzzles import (singleton_top_layer_v1_1, p2_delegated_puzzle_or_hidden_puzzle)
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')

alice balance:		2000000000000
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3


In [4]:
PUZZLE_HASH = alice.puzzle_hash
AMOUNT = 1023
MAX_VALUE = 3
VALUE = 0

singleton_terminal_puzzle = singleton_terminal_puzzle.curry(PUZZLE_HASH, AMOUNT)
singleton_counter_puzzle = counter_puzzle.curry(counter_puzzle, MAX_VALUE, singleton_terminal_puzzle, AMOUNT, VALUE)
print(singleton_terminal_puzzle.get_tree_hash())
print(singleton_counter_puzzle.get_tree_hash())

a870977635c6fb664f8fe9147e86e794ffb39a4b6e0ced1c4417a42d94249481
5105569e016af878e23bf86a02ccb75fcae1d61d6e6f0268282355b7af071dc4


In [5]:
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

inner_puzzle = singleton_counter_puzzle
standard_coin_message, coin_spends = singleton_helpers_v1_1.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = inner_puzzle,
                    keys_values = [
                        ("counter", "test")
                    ]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
launcher_id = launcher_coin_spend.coin.name()
print("\nstarting coin spend:")
singleton_helpers_v1_1.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers_v1_1.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')


starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff8203ff80ffff3dffa0

In [6]:
# sign the standard txn spend
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

# push txn to create an eve singleton
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
await network.push_tx(spend_bundle)

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: c3dff2bf57480d788070ec0e8c9bb111692cdcd751d3dae351d5747205956e47>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06

In [7]:
from chia.types.blockchain_format.program import Program
from clvm_tools.binutils import disassemble
from blspy import G2Element
async def get_last_spent_singleton_puzzle(launcher_id):
    # get the last singleton coin
    last_cr = await singleton_helpers_v1_1.get_last_singleton_coin_record(get_coin_records_by_parent_ids, launcher_id)
    coin_spend = None
    if last_cr.spent_block_index == 0:
        # if the last singleton is not spent, current count is the value of parent coin + 1
        parent_cr = await get_coin_record_by_name(last_cr.coin.parent_coin_info)
        coin_spend = await get_puzzle_and_solution(parent_cr.coin.name(), parent_cr.spent_block_index)
    else:
        # return last_cr puzzle_reveal
        coin_spend = await get_puzzle_and_solution(last_cr.coin.name(), last_cr.spent_block_index)

    if coin_spend == None:
        return None
    else:
        puzzle_reveal = coin_spend.puzzle_reveal
        return puzzle_reveal.to_program()

def get_max_value_from_curried_puzzle(puzzle):
    try:
        p = puzzle.at("rrf")     \
                    .at("rrf")   \
                    .at("rfr")   \
                    .at("rrf")   \
                    .at("rrf")   \
                    .at("rfr")
        return p.as_int()
    except Program.EvalError:
        return None

# get current count
def get_value_from_curried_puzzle(puzzle):
    try:
        p = puzzle.at("rrf")     \
                    .at("rrf")   \
                    .at("rfr")   \
                    .at("rrf")   \
                    .at("rrf")   \
                    .at("rrf")   \
                    .at("rrf")   \
                    .at("rrf")   \
                    .at("rfr")
        return p.as_int() + 1
    except Program.EvalError:
        return None


In [8]:
last_spent_singleton_puzzle = await get_last_spent_singleton_puzzle(launcher_id)
get_value_from_curried_puzzle(last_spent_singleton_puzzle)

In [9]:
launcher_id = launcher_coin_spend.coin.name()
singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

def get_count_coin_spend(singleton_coin, singleton_puzzle, coin_spend):
    lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(coin_spend)
    inner_solution = Program.to([])
    coin_spend = singleton_helpers_v1_1.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

    return coin_spend

async def count(launcher_id, prev_coin_spend):
    singleton_coin = await singleton_helpers_v1_1.get_unspent_singleton(
        get_coin_records_by_parent_ids, launcher_id)

    singleton_counter_puzzle = counter_puzzle.curry(counter_puzzle, MAX_VALUE, singleton_terminal_puzzle, AMOUNT, 0)
    singleton_puzzle = await get_last_spent_singleton_puzzle(launcher_id)
    c = get_value_from_curried_puzzle(singleton_puzzle)

    if c != None:
        singleton_counter_puzzle = counter_puzzle.curry(counter_puzzle, MAX_VALUE, singleton_terminal_puzzle, AMOUNT, c)
        
    singleton_puzzle: Program = singleton_top_layer_v1_1.puzzle_for_singleton(
            launcher_id,
            singleton_counter_puzzle # inner_puzzle
    )

    coin_spend = get_count_coin_spend(singleton_coin, singleton_puzzle, prev_coin_spend)
    result = await network.push_tx(SpendBundle(
            [coin_spend],
            G2Element()
        ))
    print(result)
    return coin_spend

In [10]:
coin_spend = await count(launcher_id, launcher_coin_spend)

{'additions': [Coin(parent_coin_info=<bytes32: 815ecd83433d842c12a2bca4ec08a9d8c795661a12abc42b5b11339a04c0ad16>, puzzle_hash=<bytes32: 57b547a80acabe515c7d41757d1a69b6120d133423781cf70e6f38a364aad244>, amount=1023)], 'removals': [Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: c3dff2bf57480d788070ec0e8c9bb111692cdcd751d3dae351d5747205956e47>, amount=1023)]}


In [11]:
puzzle = await get_last_spent_singleton_puzzle(launcher_id)
mc = get_max_value_from_curried_puzzle(puzzle)
print(mc)

c = get_value_from_curried_puzzle(puzzle)
print(c)

3
1


In [12]:
coin_spend = await count(launcher_id, coin_spend)
puzzle = await get_last_spent_singleton_puzzle(launcher_id)
c = get_value_from_curried_puzzle(puzzle)
print(c)

{'additions': [Coin(parent_coin_info=<bytes32: 9f4a5da9066c5d6c1c091d94a01e8138693f969247173c616e2910cfe4ca72b9>, puzzle_hash=<bytes32: 789db1ea2983bbf724a14d5236e272889c0d4ab13e214a16a3659f71be29174c>, amount=1023)], 'removals': [Coin(parent_coin_info=<bytes32: 815ecd83433d842c12a2bca4ec08a9d8c795661a12abc42b5b11339a04c0ad16>, puzzle_hash=<bytes32: 57b547a80acabe515c7d41757d1a69b6120d133423781cf70e6f38a364aad244>, amount=1023)]}
2


In [13]:
coin_spend = await count(launcher_id, coin_spend)
puzzle = await get_last_spent_singleton_puzzle(launcher_id)
c = get_value_from_curried_puzzle(puzzle)
print(c)

{'additions': [Coin(parent_coin_info=<bytes32: 4238e577dc4aabb5f8aff3e8ff2695123e712ac7c3670b573e150e7e03e3107b>, puzzle_hash=<bytes32: aaf3a437c916cffd74edbf605cd3d550bc1a0969f9857a212d72ed40e193a084>, amount=1023)], 'removals': [Coin(parent_coin_info=<bytes32: 9f4a5da9066c5d6c1c091d94a01e8138693f969247173c616e2910cfe4ca72b9>, puzzle_hash=<bytes32: 789db1ea2983bbf724a14d5236e272889c0d4ab13e214a16a3659f71be29174c>, amount=1023)]}
3


In [14]:
coin_spend = await count(launcher_id, coin_spend)
puzzle = await get_last_spent_singleton_puzzle(launcher_id)
c = get_value_from_curried_puzzle(puzzle)
print(c)

{'additions': [Coin(parent_coin_info=<bytes32: c82c581c59aae06554cb555f5cd406ec7987195174e8e6a7e9a95108b1058ca4>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1022)], 'removals': [Coin(parent_coin_info=<bytes32: 4238e577dc4aabb5f8aff3e8ff2695123e712ac7c3670b573e150e7e03e3107b>, puzzle_hash=<bytes32: aaf3a437c916cffd74edbf605cd3d550bc1a0969f9857a212d72ed40e193a084>, amount=1023)]}
4
